# Dose Reponse - Determine All Genes, DEGs, Unique-to-Celltype DEGS, AHR and pDREs in Deseq2 Data

## <br> 1. Import Required Packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.colors import ListedColormap

## <br> 2. Import Deseq2 Data

In [2]:
Deseq2_Directory = './Results/03d_MASTER_Deseq2_Concatonated/RDDR_Deseq2_Master_Wald_with_MGI.txt'

Deseq2_Master = pd.read_csv(Deseq2_Directory, 
                            delimiter = '\t',
                            index_col=0)

Deseq2_Master['Dose'] = Deseq2_Master['Dose'].astype('category')

Deseq2_Master = Deseq2_Master.sort_values(by='Gene')
Deseq2_Master.reset_index(drop=True, inplace=True)

Deseq2_Master

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
0,0610009B22Rik,.,LSECs,3.00,NM,0,1,3.314184,chr11,51576211,51579461,-,NaN,0.525957,1.439889,0.951191,0.552946,0.580300,NaN
1,0610009B22Rik,.,Centrilobular Hepatocytes,1.00,NM,0,1,21.595967,chr11,51576211,51579461,-,NaN,0.185332,1.137078,0.391918,0.472884,0.636296,0.951617
2,0610009B22Rik,.,LSECs,0.30,NM,0,1,3.314184,chr11,51576211,51579461,-,NaN,-1.222885,0.428425,1.390398,-0.879522,0.379119,1.000000
3,0610009B22Rik,.,Macrophages,0.10,NM,0,1,3.907435,chr11,51576211,51579461,-,NaN,0.565558,1.479960,0.709468,0.797158,0.425359,NaN
4,0610009B22Rik,.,Periportal Hepatocytes,0.03,NM,0,1,40.827277,chr11,51576211,51579461,-,NaN,0.529665,1.443593,0.304737,1.738106,0.082192,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443131,mt-Nd6,.,HSCs,0.10,mitochondrial protein-coding gene,0,0,3.672247,chrM,13551,14070,-,NaN,-0.272844,0.827686,0.836105,-0.326327,0.744177,1.000000
1443132,mt-Nd6,.,Macrophages,3.00,mitochondrial protein-coding gene,0,0,5.551356,chrM,13551,14070,-,NaN,0.692896,1.616525,0.859478,0.806182,0.420138,0.936602
1443133,mt-Nd6,.,Centrilobular Hepatocytes,10.00,mitochondrial protein-coding gene,0,0,10.872498,chrM,13551,14070,-,NaN,-0.620333,0.650521,0.635035,-0.976849,0.328644,0.558844
1443134,mt-Nd6,.,Periportal Hepatocytes,0.30,mitochondrial protein-coding gene,0,0,25.188010,chrM,13551,14070,-,NaN,-0.604237,0.657819,0.885134,-0.682651,0.494828,1.000000


In [3]:
Deseq2_Master['Celltype'].unique()

array(['LSECs', 'Centrilobular Hepatocytes', 'Macrophages',
       'Periportal Hepatocytes', 'HSCs', 'PFs', 'pDCs', 'B Cells',
       'Cholangiocytes', 'T Cells', 'Neutrophils'], dtype=object)

In [4]:
cell_types = ['Centrilobular Hepatocytes', 'Periportal Hepatocytes', 'LSECs','HSCs','Macrophages', 'B Cells','T Cells',
              'PFs', 'Cholangiocytes',  'pDCs', 'Neutrophils' ]

In [5]:
Deseq2_Master['Dose'].unique()

[3.00, 1.00, 0.30, 0.10, 0.03, 0.01, 30.00, 10.00]
Categories (8, float64): [0.01, 0.03, 0.10, 0.30, 1.00, 3.00, 10.00, 30.00]

In [6]:
doses = [0.01, 0.03, 0.10, 0.30, 1.00, 3.00, 10.00, 30.00]

### Determine total number of genes that were identified in our dataset. 

In [7]:
unique_gene_count = Deseq2_Master['Gene'].nunique()
unique_gene_count

28424

In [8]:
import os

# Define the directory path you want to check/create
directory_path = 'Results'

# Check if the directory exists
if not os.path.exists(directory_path):
    # If it doesn't exist, create the directory
    os.makedirs(directory_path)
    
    
# List of directories to check and create
directories = [
    'Results/03e_01_All_lncRNAs_In_Celltype',
    'Results/03e_02_All_DEGs_In_Celltype',
    'Results/03e_03_All_Induced_DEGs_In_Celltype',
    'Results/03e_04_All_Repressed_DEGs_In_Celltype',
    'Results/03e_05_Unique_DEGs_to_Celltypes',
    'Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR',
    'Results/03e_07_Unique_DEGs_to_Celltypes_with_DRE',
    'Results/03e_08_All_DEGs_In_Celltypes_by_Dose',
    'Results/03e_09_Induced_DEGs_In_Celltypes_by_Dose',
    'Results/03e_10_Repressed_DEGs_In_Celltypes_by_Dose',
]

# Loop through the directories
for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")
    else:
        print(f"Directory already exists: {directory}")

Directory already exists: Results/03e_01_All_lncRNAs_In_Celltype
Directory already exists: Results/03e_02_All_DEGs_In_Celltype
Directory already exists: Results/03e_03_All_Induced_DEGs_In_Celltype
Directory already exists: Results/03e_04_All_Repressed_DEGs_In_Celltype
Directory already exists: Results/03e_05_Unique_DEGs_to_Celltypes
Directory already exists: Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR
Directory already exists: Results/03e_07_Unique_DEGs_to_Celltypes_with_DRE
Directory already exists: Results/03e_08_All_DEGs_In_Celltypes_by_Dose
Directory already exists: Results/03e_09_Induced_DEGs_In_Celltypes_by_Dose
Directory already exists: Results/03e_10_Repressed_DEGs_In_Celltypes_by_Dose


## <br> 3. Determine All lncRNAs for each celltype

In [9]:
Deseq2_Master_lncRNA = Deseq2_Master[(Deseq2_Master['Type']== 'lncRNA') |
                                     (Deseq2_Master['Type']== 'antisense') |
                                     (Deseq2_Master['Type']== 'lincRNA') |
                                     (Deseq2_Master['Type']== 'NR') |
                                     (Deseq2_Master['Type']== 'lncOfInterest')]
Deseq2_Master_lncRNA

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
248,0610039H22Rik,0610039H22Rik,Periportal Hepatocytes,30.00,NR,1,0,32.438076,chr11,88510076,88514252,+,NaN,0.145682,1.106253,0.591476,0.246302,0.805449,0.882207
249,0610039H22Rik,0610039H22Rik,Periportal Hepatocytes,10.00,NR,1,0,32.438076,chr11,88510076,88514252,+,NaN,0.478944,1.393723,0.515377,0.929308,0.352730,0.590985
250,0610039H22Rik,0610039H22Rik,Centrilobular Hepatocytes,1.00,NR,1,0,13.666934,chr11,88510076,88514252,+,NaN,0.420836,1.338703,0.638845,0.658745,0.510060,0.923076
251,0610039H22Rik,0610039H22Rik,HSCs,3.00,NR,1,0,7.050317,chr11,88510076,88514252,+,NaN,0.652557,1.571952,0.900643,0.724546,0.468730,0.999835
252,0610039H22Rik,0610039H22Rik,HSCs,30.00,NR,1,0,7.050317,chr11,88510076,88514252,+,NaN,1.021181,2.029579,0.814263,1.254116,0.209800,0.500815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442435,lnc9999,Gm51874,Periportal Hepatocytes,0.03,lncRNA,0,0,8.453821,chr11,94608066,94617561,+,lnc9999,0.564477,1.478852,0.822541,0.686260,0.492549,1.000000
1442436,lnc9999,Gm51874,Periportal Hepatocytes,1.00,lncRNA,0,0,8.453821,chr11,94608066,94617561,+,lnc9999,0.162934,1.119561,0.772432,0.210936,0.832937,0.992946
1442437,lnc9999,Gm51874,Centrilobular Hepatocytes,3.00,lncRNA,0,0,4.067903,chr11,94608066,94617561,+,lnc9999,0.319668,1.248043,0.815239,0.392115,0.694973,NaN
1442438,lnc9999,Gm51874,Periportal Hepatocytes,0.10,lncRNA,0,0,8.453821,chr11,94608066,94617561,+,lnc9999,-0.236842,0.848601,0.758627,-0.312198,0.754890,1.000000


In [10]:
directory = 'Results/03e_01_All_lncRNAs_In_Celltype/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA[Deseq2_Master_lncRNA['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_All_lncRNA_Genes.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} lncRNAs")


Centrilobular Hepatocytes: 9500 lncRNAs
Periportal Hepatocytes: 12585 lncRNAs
LSECs: 4318 lncRNAs
HSCs: 3350 lncRNAs
Macrophages: 4864 lncRNAs
B Cells: 1281 lncRNAs
T Cells: 1344 lncRNAs
PFs: 366 lncRNAs
Cholangiocytes: 1252 lncRNAs
pDCs: 409 lncRNAs
Neutrophils: 257 lncRNAs


## <br> 4. Determine DEGs for each celltype

In [11]:
Deseq2_Master_lncRNA_DEGs = Deseq2_Master_lncRNA[((Deseq2_Master_lncRNA['Fold-Change'] >= 1.5) | 
                                                 (Deseq2_Master_lncRNA['Fold-Change'] <= 1/1.5)) &
                                                 (Deseq2_Master_lncRNA['padj'] <= 0.05)]
Deseq2_Master_lncRNA_DEGs

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
481,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87595571,87596153,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
484,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87590349,87592472,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
497,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87595571,87596153,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
498,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87590349,87592472,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
519,1110028F11Rik,1110028F11Rik,Periportal Hepatocytes,10.0,NR,0,5,72.459262,chr11,87590349,87592472,+,NaN,1.176435,2.260175,0.377156,3.119225,0.001813,0.014851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011
1442372,lnc9991,.,Centrilobular Hepatocytes,10.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-2.334262,0.198297,0.559680,-4.170710,0.000030,0.000400


In [12]:
directory = 'Results/03e_02_All_DEGs_In_Celltype/'


for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_DEGs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} DEGs")

Centrilobular Hepatocytes: 3463 DEGs
Periportal Hepatocytes: 3654 DEGs
LSECs: 948 DEGs
HSCs: 449 DEGs
Macrophages: 2161 DEGs
B Cells: 272 DEGs
T Cells: 377 DEGs
PFs: 45 DEGs
Cholangiocytes: 137 DEGs
pDCs: 141 DEGs
Neutrophils: 69 DEGs


## <br> 5. Determine Induced DEGs for each celltype

In [13]:
Deseq2_Master_lncRNA_Induced_DEGs = Deseq2_Master_lncRNA[(Deseq2_Master_lncRNA['Fold-Change'] >= 1.5) &
                                                 (Deseq2_Master_lncRNA['padj'] <= 0.05)]
Deseq2_Master_lncRNA_Induced_DEGs

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
481,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87595571,87596153,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
484,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87590349,87592472,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
519,1110028F11Rik,1110028F11Rik,Periportal Hepatocytes,10.0,NR,0,5,72.459262,chr11,87590349,87592472,+,NaN,1.176435,2.260175,0.377156,3.119225,0.001813,0.014851
536,1110028F11Rik,1110028F11Rik,Periportal Hepatocytes,10.0,NR,0,5,72.459262,chr11,87595571,87596153,+,NaN,1.176435,2.260175,0.377156,3.119225,0.001813,0.014851
1030,1300002E11Rik,1300002E11Rik,Centrilobular Hepatocytes,30.0,NR,0,1,183.591959,chr16,21613096,21627789,+,NaN,1.068075,2.096634,0.442546,2.413478,0.015801,0.041648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441864,lnc9959,.,T Cells,30.0,lncRNA,0,1,3.027469,chr11,86459187,86474276,+,lnc9959,3.430196,10.779332,1.263549,2.714731,0.006633,0.033859
1441880,lnc9960,.,Periportal Hepatocytes,30.0,lncRNA,0,1,20.717144,chr11,86460170,86466582,-,lnc9960,1.190020,2.281559,0.463054,2.569938,0.010172,0.032500
1442056,lnc9971,.,Macrophages,30.0,lncRNA,0,0,1.324567,chr11,87646288,87651155,-,lnc9971_Mir142b,4.352970,20.434992,1.760495,2.472583,0.013414,0.030775
1442141,lnc9975,Gm15892,Centrilobular Hepatocytes,10.0,lncRNA,0,0,7.174388,chr11,87952152,87956967,+,lnc9975_Gm15892,1.933788,3.820570,0.607605,3.182640,0.001459,0.010593


In [14]:
directory = 'Results/03e_03_All_Induced_DEGs_In_Celltype/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_Induced_DEGs[Deseq2_Master_lncRNA_Induced_DEGs['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_Induced_DEGs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} Induced DEGs")

Centrilobular Hepatocytes: 1362 Induced DEGs
Periportal Hepatocytes: 1422 Induced DEGs
LSECs: 189 Induced DEGs
HSCs: 124 Induced DEGs
Macrophages: 796 Induced DEGs
B Cells: 26 Induced DEGs
T Cells: 60 Induced DEGs
PFs: 3 Induced DEGs
Cholangiocytes: 15 Induced DEGs
pDCs: 26 Induced DEGs
Neutrophils: 15 Induced DEGs


## <br> 6. Determine Repressed DEGs for each celltype

In [15]:
Deseq2_Master_lncRNA_Repressed_DEGs = Deseq2_Master_lncRNA[(Deseq2_Master_lncRNA['Fold-Change'] <= 1/1.5) &
                                                 (Deseq2_Master_lncRNA['padj'] <= 0.05)]
Deseq2_Master_lncRNA_Repressed_DEGs

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
497,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87595571,87596153,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
498,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87590349,87592472,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
538,1110028F11Rik,1110028F11Rik,LSECs,30.0,NR,0,5,5.103935,chr11,87595571,87596153,+,NaN,-2.690600,0.154899,0.987190,-2.725515,0.006420,0.031463
539,1110028F11Rik,1110028F11Rik,LSECs,30.0,NR,0,5,5.103935,chr11,87590349,87592472,+,NaN,-2.690600,0.154899,0.987190,-2.725515,0.006420,0.031463
1180,1500035N22Rik,1500035N22Rik,LSECs,30.0,lincRNA,1,1,7.029097,chr5,25190839,25203166,+,NaN,-3.055222,0.120306,0.809917,-3.772267,0.000162,0.001405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011
1442372,lnc9991,.,Centrilobular Hepatocytes,10.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-2.334262,0.198297,0.559680,-4.170710,0.000030,0.000400


In [16]:
directory = 'Results/03e_04_All_Repressed_DEGs_In_Celltype/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_Repressed_DEGs[Deseq2_Master_lncRNA_Repressed_DEGs['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_Repressed_DEGs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} Repressed DEGs")

Centrilobular Hepatocytes: 2135 Repressed DEGs
Periportal Hepatocytes: 2252 Repressed DEGs
LSECs: 762 Repressed DEGs
HSCs: 329 Repressed DEGs
Macrophages: 1369 Repressed DEGs
B Cells: 246 Repressed DEGs
T Cells: 317 Repressed DEGs
PFs: 42 Repressed DEGs
Cholangiocytes: 122 Repressed DEGs
pDCs: 115 Repressed DEGs
Neutrophils: 54 Repressed DEGs


In [25]:
Deseq2_Master_lncRNA_DEGs_with_AHR = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['AHR'] >= 1]
Deseq2_Master_lncRNA_DEGs_with_AHR

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
481,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87595571,87596153,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
484,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87590349,87592472,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
497,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87595571,87596153,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
498,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87590349,87592472,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
519,1110028F11Rik,1110028F11Rik,Periportal Hepatocytes,10.0,NR,0,5,72.459262,chr11,87590349,87592472,+,NaN,1.176435,2.260175,0.377156,3.119225,0.001813,0.014851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442357,lnc9990,Gm35338,T Cells,30.0,lncRNA,2,5,13.375652,chr11,93998913,94053223,-,lnc9990,-2.430116,0.185551,0.546992,-4.442688,0.000009,0.000106
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011


In [26]:
directory = 'Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs_with_AHR[Deseq2_Master_lncRNA_DEGs_with_AHR['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_DEGs_with_AHR.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} lncRNA DEGs with AHR")

Centrilobular Hepatocytes: 1652 lncRNA DEGs with AHR
Periportal Hepatocytes: 1690 lncRNA DEGs with AHR
LSECs: 547 lncRNA DEGs with AHR
HSCs: 254 lncRNA DEGs with AHR
Macrophages: 1081 lncRNA DEGs with AHR
B Cells: 177 lncRNA DEGs with AHR
T Cells: 237 lncRNA DEGs with AHR
PFs: 34 lncRNA DEGs with AHR
Cholangiocytes: 98 lncRNA DEGs with AHR
pDCs: 103 lncRNA DEGs with AHR
Neutrophils: 52 lncRNA DEGs with AHR


In [28]:
Deseq2_Master_lncRNA_DEGs_with_DRE = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['pDRE'] >= 1]
Deseq2_Master_lncRNA_DEGs_with_DRE

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
1180,1500035N22Rik,1500035N22Rik,LSECs,30.0,lincRNA,1,1,7.029097,chr5,25190839,25203166,+,NaN,-3.055222,0.120306,0.809917,-3.772267,0.000162,0.001405
1187,1500035N22Rik,1500035N22Rik,Macrophages,30.0,lincRNA,1,1,15.088108,chr5,25190839,25203166,+,NaN,-2.063620,0.239215,0.615162,-3.354597,0.000795,0.002488
1211,1500035N22Rik,1500035N22Rik,Centrilobular Hepatocytes,30.0,lincRNA,1,1,163.735850,chr5,25190839,25203166,+,NaN,-1.718357,0.303895,0.531527,-3.232866,0.001226,0.004642
1603,1700016K05Rik,1700016K05Rik,LSECs,30.0,NR,1,0,3.427898,chr17,67021450,67027480,+,NaN,4.604153,24.321382,1.443398,3.189802,0.001424,0.008987
2625,1700112M02Rik,1700112M02Rik,Centrilobular Hepatocytes,10.0,lincRNA,1,0,17.566425,chr13,71635725,71760828,+,NaN,-1.202548,0.434507,0.399132,-3.012907,0.002588,0.017115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442357,lnc9990,Gm35338,T Cells,30.0,lncRNA,2,5,13.375652,chr11,93998913,94053223,-,lnc9990,-2.430116,0.185551,0.546992,-4.442688,0.000009,0.000106
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011


In [30]:
directory = 'Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs_with_DRE[Deseq2_Master_lncRNA_DEGs_with_DRE['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_DEGs_with_pDREs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} lncRNA DEGs with pDREs")

Centrilobular Hepatocytes: 1211 lncRNA DEGs with pDREs
Periportal Hepatocytes: 1292 lncRNA DEGs with pDREs
LSECs: 401 lncRNA DEGs with pDREs
HSCs: 213 lncRNA DEGs with pDREs
Macrophages: 869 lncRNA DEGs with pDREs
B Cells: 134 lncRNA DEGs with pDREs
T Cells: 177 lncRNA DEGs with pDREs
PFs: 24 lncRNA DEGs with pDREs
Cholangiocytes: 75 lncRNA DEGs with pDREs
pDCs: 79 lncRNA DEGs with pDREs
Neutrophils: 37 lncRNA DEGs with pDREs


In [31]:
Deseq2_Master_lncRNA_DEGs_with_DRE_and_AHR = Deseq2_Master_lncRNA_DEGs_with_AHR[Deseq2_Master_lncRNA_DEGs_with_AHR['pDRE'] >= 1]
Deseq2_Master_lncRNA_DEGs_with_DRE_and_AHR

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
1180,1500035N22Rik,1500035N22Rik,LSECs,30.0,lincRNA,1,1,7.029097,chr5,25190839,25203166,+,NaN,-3.055222,0.120306,0.809917,-3.772267,1.617709e-04,1.405158e-03
1187,1500035N22Rik,1500035N22Rik,Macrophages,30.0,lincRNA,1,1,15.088108,chr5,25190839,25203166,+,NaN,-2.063620,0.239215,0.615162,-3.354597,7.948060e-04,2.487993e-03
1211,1500035N22Rik,1500035N22Rik,Centrilobular Hepatocytes,30.0,lincRNA,1,1,163.735850,chr5,25190839,25203166,+,NaN,-1.718357,0.303895,0.531527,-3.232866,1.225550e-03,4.641579e-03
3208,1810058I24Rik,.,Periportal Hepatocytes,30.0,NR,1,1,130.273473,chr6,35229633,35238994,+,NaN,-1.148848,0.450985,0.374980,-3.063754,2.185785e-03,8.851795e-03
3211,1810058I24Rik,.,Centrilobular Hepatocytes,30.0,NR,1,1,127.860639,chr6,35229633,35238994,+,NaN,-2.864782,0.137282,0.413661,-6.925425,4.346671e-12,9.427542e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442357,lnc9990,Gm35338,T Cells,30.0,lncRNA,2,5,13.375652,chr11,93998913,94053223,-,lnc9990,-2.430116,0.185551,0.546992,-4.442688,8.884177e-06,1.063957e-04
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,3.701037e-03,1.386337e-02
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,1.955214e-03,1.572548e-02
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,1.307479e-06,1.066331e-05


In [32]:
directory = 'Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs_with_DRE_and_AHR[Deseq2_Master_lncRNA_DEGs_with_DRE_and_AHR['Celltype'] == cell_type]
    celltype_genes = celltype_df['Gene'].unique()
    celltype_genes_df = pd.DataFrame(celltype_genes, columns=['Gene'])
    celltype_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_lncRNA_DEGs_with_AHR_and_pDREs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(celltype_genes_df)} lncRNA DEGs with AHR and pDREs")

Centrilobular Hepatocytes: 741 lncRNA DEGs with AHR and pDREs
Periportal Hepatocytes: 770 lncRNA DEGs with AHR and pDREs
LSECs: 277 lncRNA DEGs with AHR and pDREs
HSCs: 147 lncRNA DEGs with AHR and pDREs
Macrophages: 552 lncRNA DEGs with AHR and pDREs
B Cells: 96 lncRNA DEGs with AHR and pDREs
T Cells: 127 lncRNA DEGs with AHR and pDREs
PFs: 20 lncRNA DEGs with AHR and pDREs
Cholangiocytes: 58 lncRNA DEGs with AHR and pDREs
pDCs: 66 lncRNA DEGs with AHR and pDREs
Neutrophils: 31 lncRNA DEGs with AHR and pDREs


## <br> 7. Determine DEGs that are *UNIQUE* to each celltype

In [17]:
directory = 'Results/03e_05_Unique_DEGs_to_Celltypes/'

all_genes_by_celltype = {}

# Collect all genes for each cell type
for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['Celltype'] == cell_type]
    all_genes_by_celltype[cell_type] = set(celltype_df['Gene'].unique())

# Find unique genes for each cell type
for cell_type in cell_types:
    unique_genes = all_genes_by_celltype[cell_type].copy()
    for other_type in cell_types:
        if other_type != cell_type:
            unique_genes -= all_genes_by_celltype[other_type]
    unique_genes_df = pd.DataFrame(list(unique_genes), columns=['Gene'])
    unique_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_Unique_lncRNA_DEGs.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(unique_genes_df)} unique DEGs")

Centrilobular Hepatocytes: 823 unique DEGs
Periportal Hepatocytes: 1032 unique DEGs
LSECs: 87 unique DEGs
HSCs: 62 unique DEGs
Macrophages: 604 unique DEGs
B Cells: 7 unique DEGs
T Cells: 12 unique DEGs
PFs: 4 unique DEGs
Cholangiocytes: 11 unique DEGs
pDCs: 1 unique DEGs
Neutrophils: 1 unique DEGs


## <br> 8. Determine  DEGs that are *UNIQUE* to each celltype with AHR Binding

In [18]:
Deseq2_Master_lncRNA_DEGs_with_AHR = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['AHR'] >= 1]
Deseq2_Master_lncRNA_DEGs_with_AHR

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
481,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87595571,87596153,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
484,1110028F11Rik,1110028F11Rik,Centrilobular Hepatocytes,10.0,NR,0,5,87.205582,chr11,87590349,87592472,+,NaN,1.058395,2.082613,0.406277,2.605109,0.009185,0.046380
497,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87595571,87596153,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
498,1110028F11Rik,1110028F11Rik,Macrophages,30.0,NR,0,5,6.961861,chr11,87590349,87592472,+,NaN,-3.656699,0.079291,0.783898,-4.664767,0.000003,0.000015
519,1110028F11Rik,1110028F11Rik,Periportal Hepatocytes,10.0,NR,0,5,72.459262,chr11,87590349,87592472,+,NaN,1.176435,2.260175,0.377156,3.119225,0.001813,0.014851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442357,lnc9990,Gm35338,T Cells,30.0,lncRNA,2,5,13.375652,chr11,93998913,94053223,-,lnc9990,-2.430116,0.185551,0.546992,-4.442688,0.000009,0.000106
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011


In [19]:
directory = 'Results/03e_06_Unique_DEGs_to_Celltypes_with_AHR/'

all_genes_by_celltype = {}

# Collect all genes for each cell type
for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs_with_AHR[Deseq2_Master_lncRNA_DEGs_with_AHR['Celltype'] == cell_type]
    all_genes_by_celltype[cell_type] = set(celltype_df['Gene'].unique())

# Find unique genes for each cell type
for cell_type in cell_types:
    unique_genes = all_genes_by_celltype[cell_type].copy()
    for other_type in cell_types:
        if other_type != cell_type:
            unique_genes -= all_genes_by_celltype[other_type]
    unique_genes_df = pd.DataFrame(list(unique_genes), columns=['Gene'])
    unique_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_Unique_lncRNA_DEGs_with_AHR.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(unique_genes_df)} unique DEGs with AHR")


Centrilobular Hepatocytes: 315 unique DEGs with AHR
Periportal Hepatocytes: 399 unique DEGs with AHR
LSECs: 33 unique DEGs with AHR
HSCs: 13 unique DEGs with AHR
Macrophages: 231 unique DEGs with AHR
B Cells: 1 unique DEGs with AHR
T Cells: 1 unique DEGs with AHR
PFs: 1 unique DEGs with AHR
Cholangiocytes: 4 unique DEGs with AHR
pDCs: 0 unique DEGs with AHR
Neutrophils: 0 unique DEGs with AHR


## <br> 9. Determine Repressed DEGs that are *UNIQUE* to each celltype

In [20]:
Deseq2_Master_lncRNA_DEGs_with_DRE = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['pDRE'] >= 1]
Deseq2_Master_lncRNA_DEGs_with_DRE

,Gene,MGI_Symbol,Celltype,Dose,Type,pDRE,AHR,baseMean,Chr,Start,End,Strand,NR_Annotation,log2FoldChange,Fold-Change,lfcSE,stat,pvalue,padj
1180,1500035N22Rik,1500035N22Rik,LSECs,30.0,lincRNA,1,1,7.029097,chr5,25190839,25203166,+,NaN,-3.055222,0.120306,0.809917,-3.772267,0.000162,0.001405
1187,1500035N22Rik,1500035N22Rik,Macrophages,30.0,lincRNA,1,1,15.088108,chr5,25190839,25203166,+,NaN,-2.063620,0.239215,0.615162,-3.354597,0.000795,0.002488
1211,1500035N22Rik,1500035N22Rik,Centrilobular Hepatocytes,30.0,lincRNA,1,1,163.735850,chr5,25190839,25203166,+,NaN,-1.718357,0.303895,0.531527,-3.232866,0.001226,0.004642
1603,1700016K05Rik,1700016K05Rik,LSECs,30.0,NR,1,0,3.427898,chr17,67021450,67027480,+,NaN,4.604153,24.321382,1.443398,3.189802,0.001424,0.008987
2625,1700112M02Rik,1700112M02Rik,Centrilobular Hepatocytes,10.0,lincRNA,1,0,17.566425,chr13,71635725,71760828,+,NaN,-1.202548,0.434507,0.399132,-3.012907,0.002588,0.017115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442357,lnc9990,Gm35338,T Cells,30.0,lncRNA,2,5,13.375652,chr11,93998913,94053223,-,lnc9990,-2.430116,0.185551,0.546992,-4.442688,0.000009,0.000106
1442361,lnc9991,.,Periportal Hepatocytes,30.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-2.926924,0.131495,1.008387,-2.902579,0.003701,0.013863
1442367,lnc9991,.,Periportal Hepatocytes,10.0,lncRNA,2,2,15.483385,chr11,94086841,94093423,-,lnc9991,-1.507119,0.351813,0.486646,-3.096952,0.001955,0.015725
1442370,lnc9991,.,Centrilobular Hepatocytes,30.0,lncRNA,2,2,12.060737,chr11,94086841,94093423,-,lnc9991,-6.073332,0.014851,1.255180,-4.838614,0.000001,0.000011


In [21]:
directory = 'Results/03e_07_Unique_DEGs_to_Celltypes_with_DRE/'

all_genes_by_celltype = {}

# Collect all genes for each cell type
for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs_with_DRE[Deseq2_Master_lncRNA_DEGs_with_DRE['Celltype'] == cell_type]
    all_genes_by_celltype[cell_type] = set(celltype_df['Gene'].unique())

# Find unique genes for each cell type
for cell_type in cell_types:
    unique_genes = all_genes_by_celltype[cell_type].copy()
    for other_type in cell_types:
        if other_type != cell_type:
            unique_genes -= all_genes_by_celltype[other_type]
    unique_genes_df = pd.DataFrame(list(unique_genes), columns=['Gene'])
    unique_genes_df.to_csv(f"{directory}TC_{cell_type.replace(' ', '_')}_Unique_lncRNA_DEGs_with_DRE.txt", sep='\t', header=False, index=False)
    print(f"{cell_type}: {len(unique_genes_df)} unique DEGs with DRE")


Centrilobular Hepatocytes: 232 unique DEGs with DRE
Periportal Hepatocytes: 311 unique DEGs with DRE
LSECs: 24 unique DEGs with DRE
HSCs: 20 unique DEGs with DRE
Macrophages: 219 unique DEGs with DRE
B Cells: 4 unique DEGs with DRE
T Cells: 4 unique DEGs with DRE
PFs: 3 unique DEGs with DRE
Cholangiocytes: 5 unique DEGs with DRE
pDCs: 1 unique DEGs with DRE
Neutrophils: 0 unique DEGs with DRE


## <br> 10. Determine All DEGs in each celltype by Dose

In [22]:
directory = 'Results/03e_08_All_DEGs_In_Celltypes_by_Dose/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_DEGs[Deseq2_Master_lncRNA_DEGs['Celltype'] == cell_type]
    
    for dose in doses:
        time_df = celltype_df[celltype_df['Dose'] == dose]
        genes = time_df['Gene'].unique()
        genes_df = pd.DataFrame(genes, columns=['Gene'])
        filename = f"{directory}{cell_type.replace(' ', '_')}_{dose}_All_DEGs.txt"
        genes_df.to_csv(filename, sep='\t', header=False, index=False)
        print(f"{cell_type} at Dose {dose}: {len(genes_df)} DEGs")

Centrilobular Hepatocytes at Dose 0.01: 0 DEGs
Centrilobular Hepatocytes at Dose 0.03: 3 DEGs
Centrilobular Hepatocytes at Dose 0.1: 27 DEGs
Centrilobular Hepatocytes at Dose 0.3: 48 DEGs
Centrilobular Hepatocytes at Dose 1.0: 215 DEGs
Centrilobular Hepatocytes at Dose 3.0: 382 DEGs
Centrilobular Hepatocytes at Dose 10.0: 1756 DEGs
Centrilobular Hepatocytes at Dose 30.0: 3077 DEGs
Periportal Hepatocytes at Dose 0.01: 2 DEGs
Periportal Hepatocytes at Dose 0.03: 2 DEGs
Periportal Hepatocytes at Dose 0.1: 23 DEGs
Periportal Hepatocytes at Dose 0.3: 57 DEGs
Periportal Hepatocytes at Dose 1.0: 158 DEGs
Periportal Hepatocytes at Dose 3.0: 367 DEGs
Periportal Hepatocytes at Dose 10.0: 1966 DEGs
Periportal Hepatocytes at Dose 30.0: 2982 DEGs
LSECs at Dose 0.01: 0 DEGs
LSECs at Dose 0.03: 0 DEGs
LSECs at Dose 0.1: 1 DEGs
LSECs at Dose 0.3: 1 DEGs
LSECs at Dose 1.0: 5 DEGs
LSECs at Dose 3.0: 19 DEGs
LSECs at Dose 10.0: 122 DEGs
LSECs at Dose 30.0: 895 DEGs
HSCs at Dose 0.01: 0 DEGs
HSCs at Dose 

## <br> 11. Determine Induced DEGs in each celltype by Dose

In [23]:
directory = 'Results/03e_09_Induced_DEGs_In_Celltypes_by_Dose/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_Induced_DEGs[Deseq2_Master_lncRNA_Induced_DEGs['Celltype'] == cell_type]
    
    for dose in doses:
        time_df = celltype_df[celltype_df['Dose'] == dose]
        genes = time_df['Gene'].unique()
        genes_df = pd.DataFrame(genes, columns=['Gene'])
        filename = f"{directory}{cell_type.replace(' ', '_')}_{dose}_Induced_DEGs.txt"
        genes_df.to_csv(filename, sep='\t', header=False, index=False)
        print(f"{cell_type} at Dose {dose}: {len(genes_df)} Induced DEGs")

Centrilobular Hepatocytes at Dose 0.01: 0 Induced DEGs
Centrilobular Hepatocytes at Dose 0.03: 3 Induced DEGs
Centrilobular Hepatocytes at Dose 0.1: 25 Induced DEGs
Centrilobular Hepatocytes at Dose 0.3: 43 Induced DEGs
Centrilobular Hepatocytes at Dose 1.0: 170 Induced DEGs
Centrilobular Hepatocytes at Dose 3.0: 240 Induced DEGs
Centrilobular Hepatocytes at Dose 10.0: 612 Induced DEGs
Centrilobular Hepatocytes at Dose 30.0: 1178 Induced DEGs
Periportal Hepatocytes at Dose 0.01: 2 Induced DEGs
Periportal Hepatocytes at Dose 0.03: 2 Induced DEGs
Periportal Hepatocytes at Dose 0.1: 22 Induced DEGs
Periportal Hepatocytes at Dose 0.3: 45 Induced DEGs
Periportal Hepatocytes at Dose 1.0: 138 Induced DEGs
Periportal Hepatocytes at Dose 3.0: 278 Induced DEGs
Periportal Hepatocytes at Dose 10.0: 722 Induced DEGs
Periportal Hepatocytes at Dose 30.0: 1147 Induced DEGs
LSECs at Dose 0.01: 0 Induced DEGs
LSECs at Dose 0.03: 0 Induced DEGs
LSECs at Dose 0.1: 1 Induced DEGs
LSECs at Dose 0.3: 1 Induc

## <br> 12. Determine Repressed DEGs in each celltype by Dose

In [24]:
Deseq2_Master_lncRNA_Repressed_DEGs

directory = 'Results/03e_10_Repressed_DEGs_In_Celltypes_by_Dose/'

for cell_type in cell_types:
    celltype_df = Deseq2_Master_lncRNA_Repressed_DEGs[Deseq2_Master_lncRNA_Repressed_DEGs['Celltype'] == cell_type]
    
    for dose in doses:
        time_df = celltype_df[celltype_df['Dose'] == dose]
        genes = time_df['Gene'].unique()
        genes_df = pd.DataFrame(genes, columns=['Gene'])
        filename = f"{directory}{cell_type.replace(' ', '_')}_{dose}_Repressed_DEGs.txt"
        genes_df.to_csv(filename, sep='\t', header=False, index=False)
        print(f"{cell_type} at Dose {dose}: {len(genes_df)} Repressed DEGs")

Centrilobular Hepatocytes at Dose 0.01: 0 Repressed DEGs
Centrilobular Hepatocytes at Dose 0.03: 0 Repressed DEGs
Centrilobular Hepatocytes at Dose 0.1: 2 Repressed DEGs
Centrilobular Hepatocytes at Dose 0.3: 5 Repressed DEGs
Centrilobular Hepatocytes at Dose 1.0: 45 Repressed DEGs
Centrilobular Hepatocytes at Dose 3.0: 142 Repressed DEGs
Centrilobular Hepatocytes at Dose 10.0: 1144 Repressed DEGs
Centrilobular Hepatocytes at Dose 30.0: 1899 Repressed DEGs
Periportal Hepatocytes at Dose 0.01: 0 Repressed DEGs
Periportal Hepatocytes at Dose 0.03: 0 Repressed DEGs
Periportal Hepatocytes at Dose 0.1: 1 Repressed DEGs
Periportal Hepatocytes at Dose 0.3: 12 Repressed DEGs
Periportal Hepatocytes at Dose 1.0: 20 Repressed DEGs
Periportal Hepatocytes at Dose 3.0: 89 Repressed DEGs
Periportal Hepatocytes at Dose 10.0: 1244 Repressed DEGs
Periportal Hepatocytes at Dose 30.0: 1835 Repressed DEGs
LSECs at Dose 0.01: 0 Repressed DEGs
LSECs at Dose 0.03: 0 Repressed DEGs
LSECs at Dose 0.1: 0 Repress